Fundamentos de ciência de dados - Luiz Davi e Thiago Cavalcanti

In [1]:
from nba_api_utils import obter_jogos_por_time_e_temporada, salvar_dataset_csv
from constantes import NOME_TIME, TIME_ID, TEMPORADA_ANTERIOR, TEMPORADA_ATUAL, RESULTADOS_PATH
from nba_api.stats.endpoints import boxscoretraditionalv2
import pandas as pd

# RF7 – Apresentar os jogos do seu time [temporada 23-24 e temporada atual], conforme Tabela 6.

In [2]:
def obter_apresentacao_jogos(dados_jogos):
    jogos_formatados = []

    for indice, linha in dados_jogos.iterrows():
        data_jogo = linha['GAME_DATE']
        adversario = linha['MATCHUP'][-3:].strip()
        if linha['WL'] is not None:
            resultado = 'Vitória' if 'W' in linha['WL'] else 'Derrota'
        else:
            resultado = 'Indefinido'
        casa_ou_fora = 'Casa' if '@' not in linha['MATCHUP'] else 'Fora'
    
        jogos_formatados.append([data_jogo, adversario, resultado, casa_ou_fora])

    estatisticas = pd.DataFrame(jogos_formatados, columns=['Data do jogo', 'Adversário', 'Vitória ou derrota', 'Casa ou fora'])
    
    return estatisticas

In [3]:
def adicionar_placares(dados_jogos, estatisticas):
    placares = []
    for indice, linha in dados_jogos.iterrows():
        game_id = linha['Game_ID']
        jogo_boxscore = boxscoretraditionalv2.BoxScoreTraditionalV2(game_id=game_id)
        dados_boxscore = jogo_boxscore.get_data_frames()[0]

        if '@' not in linha['MATCHUP']:
            placar_time = linha['PTS']
            placar_adversario = dados_boxscore[dados_boxscore['TEAM_ID'] != TIME_ID]['PTS'].sum()
        else:
            placar_adversario = linha['PTS']
            placar_time = dados_boxscore[dados_boxscore['TEAM_ID'] != TIME_ID]['PTS'].sum()
        
        placar = f'{int(placar_time)} - {int(placar_adversario)}'
        placares.append(placar)
    
    estatisticas['Placar'] = placares

## Estatísticas da temporada anterior

In [4]:
dados_jogos_passados = obter_jogos_por_time_e_temporada(NOME_TIME, TEMPORADA_ANTERIOR)
estatisticas_passadas = obter_apresentacao_jogos(dados_jogos_passados)

In [15]:
adicionar_placares(dados_jogos_passados, estatisticas_passadas)

salvar_dataset_csv(estatisticas_passadas, f"{RESULTADOS_PATH}/rf7_estatisticas_temporada_passada.csv")

estatisticas_passadas

,Data do jogo,Adversário,Vitória ou derrota,Casa ou fora,Placar
0,"APR 14, 2024",MEM,Vitória,Fora,111 - 126
1,"APR 12, 2024",SAS,Derrota,Fora,121 - 120
2,"APR 10, 2024",MIN,Vitória,Casa,116 - 107
3,"APR 09, 2024",UTA,Vitória,Fora,95 - 111
4,"APR 06, 2024",ATL,Vitória,Casa,142 - 110
...,...,...,...,...,...
77,"NOV 01, 2023",MIN,Derrota,Fora,110 - 89
78,"OCT 30, 2023",UTA,Vitória,Casa,110 - 102
79,"OCT 29, 2023",OKC,Vitória,Fora,95 - 128
80,"OCT 27, 2023",MEM,Vitória,Fora,104 - 108


## Estatísticas da temporada atual

In [5]:
dados_jogos_atuais = obter_jogos_por_time_e_temporada(NOME_TIME, TEMPORADA_ATUAL)
estatisticas_atuais = obter_apresentacao_jogos(dados_jogos_atuais)

In [14]:
adicionar_placares(dados_jogos_atuais, estatisticas_atuais)

salvar_dataset_csv(estatisticas_passadas, f"{RESULTADOS_PATH}/rf7_estatisticas_temporada_atual.csv")

estatisticas_atuais

,Data do jogo,Adversário,Vitória ou derrota,Casa ou fora,Placar
0,"JAN 17, 2025",MIA,Vitória,Fora,113 - 133
1,"JAN 15, 2025",HOU,Derrota,Casa,108 - 128
2,"JAN 14, 2025",DAL,Vitória,Fora,99 - 118
3,"JAN 12, 2025",DAL,Vitória,Fora,101 - 112
4,"JAN 10, 2025",BKN,Vitória,Casa,124 - 105
5,"JAN 08, 2025",LAC,Vitória,Casa,126 - 103
6,"JAN 07, 2025",BOS,Derrota,Casa,106 - 118
7,"JAN 04, 2025",SAS,Vitória,Fora,111 - 122
8,"JAN 03, 2025",SAS,Derrota,Casa,110 - 113
9,"JAN 01, 2025",ATL,Vitória,Casa,139 - 120
